<a href="https://colab.research.google.com/github/nhso/eng-nsho-scraping/blob/Version-0.2-Update-fine-pdf-URLs/%E0%B8%AA%E0%B8%9B%E0%B8%AA%E0%B8%8A_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [4]:
#url = ["http://eng.nhso.go.th/view/1/News/EN-US/?page=", "http://eng.nhso.go.th/view/1/In_Depth/EN-US/?page=", "http://eng.nhso.go.th/view/1/Events/EN-US"]

url = "http://eng.nhso.go.th/view/1/News/EN-US/?page="
page = 1
new_urls = []

while page <= 10 :
  data = requests.get(url + str(page))
  soup = BeautifulSoup(data.text,'html.parser')
  page += 1
  for i in range(1, 12) :
    ne = soup.find_all("a", {'class' : 'title-7'})[i]
    new_urls.append(ne)

url = "http://eng.nhso.go.th/view/1/In_Depth/EN-US/?page="
page = 1
In_Depth_urls = []

while page <= 11 :
  data = requests.get(url + str(page))
  soup = BeautifulSoup(data.text,'html.parser')
  page += 1
  for i in range(1, 8) :
    IDep = soup.find_all("a", {'href' : 'javascript:;'})[i]
    new_urls.append(IDep)

In [5]:
string_list = [str(element) for element in new_urls]
new_urls = pd.DataFrame(string_list, columns=['Raw'])
new_urls

df_new = new_urls['Raw'].str.split("'", expand=True)
df_new.columns = ['Column1','URLs','Column1','Column1','Column1','Column1','Column1','Column1','Column1','Column1','Column1','Column1','Column1','Column1','Column1']
columns_drop = ['Column1']
news_urls = df_new.drop(columns_drop, axis=1)

news_urls

,URLs
0,https://eng.nhso.go.th/view/1/DescriptionNews/...
1,https://eng.nhso.go.th/view/1/DescriptionNews/...
2,https://eng.nhso.go.th/view/1/DescriptionNews/...
3,https://eng.nhso.go.th/view/1/DescriptionNews/...
4,https://eng.nhso.go.th/view/1/DescriptionNews/...
...,...
182,https://eng.nhso.go.th/view/1/DescriptionNews/...
183,https://eng.nhso.go.th/view/1/DescriptionNews/...
184,https://eng.nhso.go.th/view/1/DescriptionNews/...
185,https://eng.nhso.go.th/view/1/DescriptionNews/...


In [6]:
url = ["http://eng.nhso.go.th/view/1/Annual_Reports/EN-US","http://eng.nhso.go.th/view/1/UHC_Public_Relations/EN-US","http://eng.nhso.go.th/view/1/Articles/EN-US","http://eng.nhso.go.th/view/1/Books/EN-US","http://eng.nhso.go.th/view/1/Leaflets/EN-US"
, "http://eng.nhso.go.th/view/1/Books/EN-US/?page=2", "http://eng.nhso.go.th/view/1/Books/EN-US/?page=3", "http://eng.nhso.go.th/view/1/Books/EN-US/?page=4", "http://eng.nhso.go.th/view/1/Books/EN-US/?page=5", "http://eng.nhso.go.th/view/1/Reports/EN-US"
, "http://eng.nhso.go.th/view/1/Reports/EN-US/?page=2", "http://eng.nhso.go.th/view/1/Companion_Book/EN-US"]

pdf_url = []
pdf_name = []
for i in url:
  data = requests.get(i)
  soup = BeautifulSoup(data.text)
  for j in soup.find_all("a",style="text-align:left;",href="javascript:;") :
    pdf_name.append(j.text)
    pdf_url.append(j)

table = pd.DataFrame([pdf_name, pdf_url]).transpose()
table.columns = ['Title', 'DATA']
table['DATA'] = table['DATA'].astype(str)
df_new = table['DATA'].str.split("'", expand=True)
df_new.columns = ['Column1','URLs','Column1','Column1','Column1','Column1','Column1','Column1','Column1','Column1']
pdf_only_urls = df_new.drop(columns_drop, axis=1)
pdf_only_urls['URLs'] = 'http://eng.nhso.go.th' + pdf_only_urls['URLs']
pdf_only_urls = pdf_only_urls.drop(index=49)
pdf_only_urls = pdf_only_urls.reset_index().drop(columns='index')

#pdf_only_urls

In [7]:
url = ["http://eng.nhso.go.th/view/1/Philosophy_Background/EN-US","http://eng.nhso.go.th/view/1/NHSO_Regional_Office/EN-US","http://eng.nhso.go.th/view/1/National_Health_Security_Act_B.E.2545/EN-US","http://eng.nhso.go.th/view/1/Contact_NHSO/EN-US"]

body = []

for i in url:
  data = requests.get(i)
  soup = BeautifulSoup(data.text,)
  for j in soup.find_all("div",{"id":"content-B"}) :
    if "titel-head" in j.get("class" ,[]) :
      continue
    body.append(j.text)

table = pd.DataFrame(body)
table.columns = ['DATA']

#table

In [8]:
url = ["http://eng.nhso.go.th/view/1/Vision_Mission/EN-US", "http://eng.nhso.go.th/view/1/NHSO_Board/EN-US", "http://eng.nhso.go.th/view/1/NHSO_Executive/EN-US",
       "http://eng.nhso.go.th/view/1/NHSB/EN-US", "http://eng.nhso.go.th/view/1/QSC/EN-US"]

for i in url:
  data = requests.get(i)
  soup = BeautifulSoup(data.text,)
  for j in soup.find_all("div",{"id":"content-A"}) :
    if "page-tree" in j.get("class" ,[]) :
      continue
    body.append(j.text)

table = pd.DataFrame(body)
table.columns = ['DATA']

#table